In [1]:
%load_ext literary.module

# Notebook Finder

In [1]:
import os
import sys
from importlib.machinery import FileFinder, PathFinder
from inspect import getclosurevars

The notebook loader needs to be installed by modifying the existing `FileFinder`. This is so that packages can be loaded with `__init__.ipynb` or `__init__.py` modules.

In [2]:
def extend_file_finder(*loader_details):
    """Inject a set of loaders into a list of path hooks

    :param path_hooks: list of path hooks
    :param loader_details: FileFinder loader details
    :return:
    """
    for i, hook in enumerate(sys.path_hooks):
        try:
            namespace = getclosurevars(hook)
        except TypeError as err:
            continue

        try:
            details = namespace.nonlocals["loader_details"]
        except KeyError as err:
            continue

        break
    else:
        raise ValueError

    sys.path_hooks[i] = FileFinder.path_hook(*details, *loader_details)

    # To fix cached path finders
    sys.path_importer_cache.clear()

We will want to lazily load the notebook loader factory so that the Python startup is not adversely affected by importing nbconvert et al. However, IPyKernel puts the working directory on the `sys.path` by default, which causes problems if any notebooks shadow built-in packages. Although this also holds for regular Python files in the working directory, in practice this behaviour is not desirable, so we choose to (opt-out) remove the working directory from the path using a meta path finder.

In [3]:
class IPyKernelPathRestrictor:
    @classmethod
    def find_spec(cls, fullname, path=None, target=None):
        if "." in fullname:
            return None

        name = fullname.split(".", 1)[0]

        # IPython controls the kernel startup including its path
        # The user cannot easily shape this, so instead we ignore notebooks
        # during initialisation, to avoid problems of name shadowing in the cwd
        if name == "ipykernel":
            cwd = os.path.realpath(os.getcwd())
            sys.path = [p for p in sys.path if os.path.realpath(p) != cwd]

        return None

Let's make a dedicated installer for this

In [4]:
def install_ipykernel_restrictor():
    for i, finder in enumerate(sys.meta_path):
        if finder is PathFinder:
            break
    else:
        return

    sys.meta_path.insert(i, IPyKernelPathRestrictor)